In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
data = pd.read_csv('./movielens_sample.csv')
sparse_features = ['movie_id','user_id','gender','age','occupation','zip']
target = ['rating']

In [3]:
data.head()

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,3299,235,4,968035345,Ed Wood (1994),Comedy|Drama,F,25,4,19119
1,3630,3256,3,966536874,Patriot Games (1992),Action|Thriller,M,18,4,77005
2,517,105,4,976203603,"Bridges of Madison County, The (1995)",Drama|Romance,F,25,14,55408
3,785,2115,3,975430389,Indiana Jones and the Temple of Doom (1984),Action|Adventure,M,18,19,29307
4,5848,909,5,957782527,"Apartment, The (1960)",Comedy|Drama,M,50,20,20009


In [4]:
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])

In [10]:
from deepctr.feature_column import SparseFeat,get_feature_names

In [8]:
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique()) for feature in sparse_features]

In [9]:
fixlen_feature_columns

[SparseFeat(name='movie_id', vocabulary_size=187, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.initializers_v1.RandomNormal object at 0x7fd5933d6610>, embedding_name='movie_id', group_name='default_group', trainable=True),
 SparseFeat(name='user_id', vocabulary_size=193, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.initializers_v1.RandomNormal object at 0x7fd5933d66d0>, embedding_name='user_id', group_name='default_group', trainable=True),
 SparseFeat(name='gender', vocabulary_size=2, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.initializers_v1.RandomNormal object at 0x7fd5933d6750>, embedding_name='gender', group_name='default_group', trainable=True),
 SparseFeat(name='age', vocabulary_size=7, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initiali

#### 其实我们不需要进行特征名称的转换 因为我们line 和knn 的输入特征是一样的

In [13]:
feature_names = get_feature_names(fixlen_feature_columns)

In [14]:
feature_names

['movie_id', 'user_id', 'gender', 'age', 'occupation', 'zip']

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
train, test = train_test_split(data, test_size=0.2)

train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}


In [17]:
from deepctr.models import DeepFM

In [20]:
model = DeepFM(fixlen_feature_columns,fixlen_feature_columns,task='regression')
model.compile("adam", "mse", metrics=['mse'])
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=1, verbose=True, validation_split=0.2)

/Users/caoyongjun/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1/1 [==============================] - 0s 248ms/step - loss: 13.8895 - mse: 13.8895 - val_loss: 15.6336 - val_mse: 15.6336


In [21]:
history

In [22]:
pred_ans = model.predict(test_model_input, batch_size=256)

In [23]:
from sklearn.metrics import mean_squared_error

In [24]:
mse = mean_squared_error(test[target].values,pred_ans)

In [25]:
mse

13.542171422042419

In [26]:
mse ** 0.5

3.6799689430812346

### 没有找到完整的数据集